Author: Kevin ALBERT

Created: Nov 2020

In [1]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from IPython.display import Javascript
# from fuzzywuzzy import process, fuzz

In [2]:
import platform
import psutil
import os

In [3]:
conda_version = ! conda -V
print(f"conda : {conda_version[0].split()[1]}")
pip_version = ! pip -V
print(f"pip   : {pip_version[0].split()[1]}")
python_version = ! python -V
print(f"python: {python_version[0].split()[1]}")
pandas_version = ! pip list |grep -i pandas
print(f"pandas: {pandas_version[0].split()[1]}")
py2neo_version = ! pip list |grep -i py2neo
print(f"py2neo: {py2neo_version[0].split()[1]}")

conda : 4.9.2
pip   : 20.2.4
python: 3.8.6
pandas: 1.1.4
py2neo: 4.2.0


In [29]:
server  = "13.93.37.217"
port    = "7687"
user    = "neo4j"
passw   = "digityser"
db_name = "neo4j"        # default name (v4.x)

In [30]:
# load graph connection instance
graph = Graph(host=server, auth=(user, passw), name=db_name, encrypted=False)

### prepare data

In [6]:
# import the dataset
synthetic_df = pd.read_parquet("../../data/silver/synthetic_data_processed.parquet")

In [ ]:
# creating unique patient ID
synthetic_df["id"] = synthetic_df.index
# real_df["id"] = real_df.index

In [7]:
meds_list = ['metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide_metformin', 'glipizide_metformin',
       'glimepiride_pioglitazone', 'metformin_rosiglitazone',
       'metformin_pioglitazone']
for meds in meds_list:
    synthetic_df[meds+"_medicine"] = synthetic_df[meds].apply(lambda x: 'None' if (x=='No') else meds)

In [21]:
synthetic_df['readmitted'].apply(lambda x: 'None' if (x=='No') else meds)

0        <30
1         NO
2        >30
3        >30
4        >30
        ... 
78436    >30
78437    <30
78438     NO
78439    >30
78440     NO
Name: readmitted, Length: 78441, dtype: object

In [27]:
# we make severe, normal diabetes severity levels
hba1c = {'None':'None', 'Norm':'Normal', '>8':'Severe_Diabetes', '>7':'Diabetes'}
synthetic_df["A1Cresult"] = synthetic_df["A1Cresult"].map(hba1c)

In [28]:
# save file to /import
file = "synthetic_data_processed.csv"
synthetic_df.to_csv("../../neo4j/import/"+file, sep=',', index=False)

### import data in DB

In [10]:
graph.run("CREATE CONSTRAINT ON (p:Patient) ASSERT p.id IS UNIQUE")

In [11]:
graph.run("CREATE CONSTRAINT ON (r:Race) ASSERT r.value IS UNIQUE")

In [12]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (p:Patient {id:(line.id)})
MERGE (r:Race {value:(line.race)})
MERGE (p)-[:HAS_RACE]->(r)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 78446
labels_removed: 0
nodes_created: 78446
nodes_deleted: 0
properties_set: 78446
relationships_created: 78441
relationships_deleted: 0

In [13]:
graph.run("CREATE CONSTRAINT ON (g:Gender) ASSERT g.value IS UNIQUE")

In [14]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (p:Patient {id:(line.id)})
MERGE (g:Gender {value:(line.gender)})
MERGE (p)-[:HAS_GENDER]->(g)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 2
labels_removed: 0
nodes_created: 2
nodes_deleted: 0
properties_set: 2
relationships_created: 78441
relationships_deleted: 0

In [15]:
graph.run("CREATE CONSTRAINT ON (ag:AgeGroup) ASSERT ag.value IS UNIQUE")

In [16]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (p:Patient {id:(line.id)})
MERGE (ag:AgeGroup {value:(line.age)})
MERGE (p)-[:HAS_AGE]->(ag)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 10
labels_removed: 0
nodes_created: 10
nodes_deleted: 0
properties_set: 10
relationships_created: 78441
relationships_deleted: 0

In [17]:
# graph.run("CREATE CONSTRAINT ON (m:Medicine) ASSERT m.value IS UNIQUE")
graph.run("CREATE INDEX ON :Medicine(value)")

In [18]:
for meds in meds_list:
    print(meds)
    query = """
    USING PERIODIC COMMIT 1000
    LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
    WITH line.id AS id, line."""+meds+"""_medicine AS medicine, line."""+meds+""" AS drugdose
    WHERE medicine <> "None" AND drugdose <> "No"
    MATCH (p:Patient {id:(id)})
    MERGE (m:Medicine {value:(medicine), drugdose:(drugdose)})
    MERGE (p)-[:USE_MEDICINE]->(m)
    """
    display(graph.run(query).stats())

metformin


constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 3
labels_removed: 0
nodes_created: 3
nodes_deleted: 0
properties_set: 6
relationships_created: 15593
relationships_deleted: 0

repaglinide


constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 3
labels_removed: 0
nodes_created: 3
nodes_deleted: 0
properties_set: 6
relationships_created: 1723
relationships_deleted: 0

nateglinide


...

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 1
labels_removed: 0
nodes_created: 1
nodes_deleted: 0
properties_set: 2
relationships_created: 113
relationships_deleted: 0

metformin_rosiglitazone


constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

metformin_pioglitazone


constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

In [19]:
# graph.run("CREATE CONSTRAINT ON (hd:HospitalDays) ASSERT hd.severity IS UNIQUE")
graph.run("CREATE INDEX ON :HospitalDays(severity)")

In [20]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (p:Patient {id:(line.id)})
MERGE (hd:HospitalDays {severity:(line.time_in_hospital_severitylvl), value:(line.time_in_hospital)})
MERGE (p)-[:HOSPITAL_STAY]->(hd)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 14
labels_removed: 0
nodes_created: 14
nodes_deleted: 0
properties_set: 28
relationships_created: 78441
relationships_deleted: 0

In [31]:
graph.run("CREATE CONSTRAINT ON (h:HbA1c) ASSERT h.value IS UNIQUE")

In [34]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
WITH line
WHERE line.A1Cresult <> "None"
MATCH (p:Patient {id:(line.id)})
MERGE (h:HbA1c {value:(line.A1Cresult)})
MERGE (p)-[:HBA1C_STATUS]->(h)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 3
labels_removed: 0
nodes_created: 3
nodes_deleted: 0
properties_set: 3
relationships_created: 11634
relationships_deleted: 0

In [35]:
graph.run("CREATE CONSTRAINT ON (ra:Readmitted) ASSERT ra.value IS UNIQUE")

In [37]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (p:Patient {id:(line.id)})
MERGE (ra:Readmitted {value:(line.readmitted)})
MERGE (p)-[:RECURRENT]->(ra)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 3
labels_removed: 0
nodes_created: 3
nodes_deleted: 0
properties_set: 3
relationships_created: 78441
relationships_deleted: 0

## Querying

In [ ]:
MATCH (n:HbA1c)<-[:HBA1C_STATUS]-(p:Patient)-[:USE_MEDICINE]->(m:Medicine)
WHERE m.value='insulin' AND m.drugdose='Down'
WITH m, p
MATCH (p)-[:HAS_AGE]->(ag:AgeGroup)
WHERE ag.value='80-90'
WITH p, ag
MATCH (n:HbA1c)<-[:HBA1C_STATUS]-(p)-[:USE_MEDICINE]->(m2:Medicine)
MATCH (p)-[:RECURRENT]->(ra:Readmitted)
WHERE m2.drugdose='Down' OR m2.drugdose='Steady' 
RETURN * ORDER BY ag.value DESC LIMIT 200

In [42]:
%%time
query = """
MATCH (n:HbA1c)<-[:HBA1C_STATUS]-(p:Patient)-[:USE_MEDICINE]->(m:Medicine)
WHERE m.value='insulin' AND m.drugdose='Down'
WITH m, p
MATCH (p)-[:HAS_AGE]->(ag:AgeGroup)
WHERE ag.value='80-90'
WITH p, ag
MATCH (n:HbA1c)<-[:HBA1C_STATUS]-(p)-[:USE_MEDICINE]->(meds:Medicine)
MATCH (p)-[:RECURRENT]->(ra:Readmitted)
//WHERE meds.drugdose='Down' OR meds.drugdose='Steady' 
RETURN ag.value, meds, n.value, COLLECT(DISTINCT p.id), ra.value ORDER BY ag.value DESC LIMIT 200
"""
graph.run(query).to_data_frame()

CPU times: user 7.19 ms, sys: 0 ns, total: 7.19 ms
Wall time: 208 ms


,ag.value,meds,n.value,COLLECT(DISTINCT p.id),ra.value
0,80-90,"{'drugdose': 'Down', 'value': 'insulin'}",Diabetes,"[78354, 76451, 72542, 70607, 70436, 70990, 698...",NO
1,80-90,"{'drugdose': 'Steady', 'value': 'metformin'}",Diabetes,"[78354, 70607, 64366, 36449, 35523, 17308, 156...",NO
2,80-90,"{'drugdose': 'Down', 'value': 'insulin'}",Normal,"[78276, 77890, 77268, 74962, 74254, 74201, 743...",>30
3,80-90,"{'drugdose': 'Steady', 'value': 'glimepiride'}",Normal,"[78276, 77268, 70722]",>30
4,80-90,"{'drugdose': 'Steady', 'value': 'metformin'}",Normal,"[78276, 77268, 74254, 51548, 45795, 42139, 242...",>30
...,...,...,...,...,...
108,80-90,"{'drugdose': 'Up', 'value': 'pioglitazone'}",Severe_Diabetes,[5959],NO
109,80-90,"{'drugdose': 'Steady', 'value': 'glyburide'}",Diabetes,[2719],NO
110,80-90,"{'drugdose': 'Steady', 'value': 'pioglitazone'}",Normal,[2310],NO
111,80-90,"{'drugdose': 'Steady', 'value': 'chlorpropamide'}",Severe_Diabetes,[1899],>30


### Can you explore which treatments are effective, and which ones are not?

### What are the most important factors for hospital readmission?

### Is the best cure insulin or a combination of drugs?

### What's the best treatment for a newly diagnosed patient based on patient's features and the response to certain drugs?

### reset DB

In [ ]:
# delete database and restart (wait ~2min)
import os
os.system(" cd ../.. && \
            sudo docker-compose down && \
            sudo rm -Rf neo4j/data/databases/neo4j && \
            sudo rm -Rf neo4j/data/transactions/neo4j && \
            sudo docker-compose up --build &")